In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#필요 라이브러리 임포트
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings(action='ignore')
plt.rcParams['axes.unicode_minus'] = False
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder

# interactive graph
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
# Any results you write to the current directory are saved as output.

/kaggle/input/home-credit-default-risk/previous_application.csv
/kaggle/input/home-credit-default-risk/installments_payments.csv
/kaggle/input/home-credit-default-risk/credit_card_balance.csv
/kaggle/input/home-credit-default-risk/application_train.csv
/kaggle/input/home-credit-default-risk/bureau.csv
/kaggle/input/home-credit-default-risk/application_test.csv
/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv
/kaggle/input/home-credit-default-risk/bureau_balance.csv
/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv
/kaggle/input/home-credit-default-risk/sample_submission.csv


In [2]:
train = pd.read_csv('/kaggle/input/home-credit-default-risk/application_train.csv')
test = pd.read_csv('/kaggle/input/home-credit-default-risk/application_test.csv')
result = pd.read_csv('/kaggle/input/home-credit-default-risk/sample_submission.csv')

In [3]:
print(train.shape)
print(test.shape)

(307511, 122)
(48744, 121)


In [4]:
train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
def data_eda(train):
    fig = px.bar(train,title = 'feature 별 null 값 갯수', x=train.isnull().sum().sort_values(ascending = False).index, y=train.isnull().sum().sort_values(ascending = False) / len(train), text = train.isnull().sum().sort_values(ascending = False) / len(train),height= 500)
    fig.show()

def data_target_plot(train, target):
    fig = px.bar(train,title = '타겟 비율 확인', x=[0,1], y=train[target].value_counts() / len(train), text = (train[target].value_counts() / len(train)).round(2), color=train[target].value_counts(),color_continuous_scale=['green','red'],width=600, height= 500)
    fig.show()

data_eda(train)
data_target_plot(train, 'TARGET')
data_eda(test)

In [6]:
def show_plot1(columns):
    print(columns)
    
    train[columns] = pd.cut(train[columns],10,labels=False)
    sns.catplot(x=columns, col='TARGET', kind='count',data = train)
    
    train[columns] = train[columns].apply(lambda x :0 if np.isnan(x) else 1)
    sns.catplot(x=columns, col='TARGET', kind='count', data=train)
    plt.show()
    
def show_plot2(columns):
    print(columns)
    # class 별 생존자 확인
    sns.catplot(x=columns, col='TARGET', kind='count', data=train)

def show_plot3(columns):
    train[columns] = train[columns].fillna('F').apply(lambda x : 0 if x[0] == 'F' else 1)
    sns.catplot(x=columns, col='TARGET', kind='count', data=train)
    

In [7]:
# for i in null_columns:
#     if train[i].dtype == 'object': # float64
#         show_plot2(i)     

In [8]:
# for i in null_columns:
#     print(train[i].dtype)
#     if train[i].dtype == 'float64' or train[i].dtype == 'int64': # float64
#         show_plot1(i)

In [9]:
lst = ['EMERGENCYSTATE_MODE', # 그냥 / 값 유무
       'HOUSETYPE_MODE', # 값 유무
       'OCCUPATION_TYPE', # 값 유무
       'ENTRANCES_MODE', # 값 유무
       'FLOORMAX_AVG', # 값 유무
       'FLOORMAX_MEDI', # 값 유무
       'FLOORMAX_MODE', # 값 유무
       'LIVINGAREA_AVG', # 값 유무
       'LIVINGAREA_MEDI', # 값 유무
       'LIVINGAREA_MODE', # 값 유무
       'TOTALAREA_MODE',
       'YEARS_BEGINEXPLUATATION_AVG',
       'YEARS_BEGINEXPLUATATION_MEDI',
       'YEARS_BEGINEXPLUATATION_MODE'
      ]

In [10]:
def data_processing(train):
    null_columns = sorted(list(train.isnull().sum().sort_values(ascending = False).index[:57]))

    new_columns = []

    for i in train.columns:
        if i not in null_columns:
            new_columns.append(i)
    
    train = train[new_columns]
    
    print(train.shape)
    
    train['NAME_TYPE_SUITE'].fillna('NaN', inplace = True)
    train['DEF_60_CNT_SOCIAL_CIRCLE'].fillna(0, inplace = True)
    train['OBS_60_CNT_SOCIAL_CIRCLE'].fillna(0, inplace = True)
    train['DEF_30_CNT_SOCIAL_CIRCLE'].fillna(0, inplace = True)
    train['OBS_30_CNT_SOCIAL_CIRCLE'].fillna(0, inplace = True)
    
    train['EXT_SOURCE_2'].fillna(0, inplace = True) # 값을 무엇으로 채울지
    train['AMT_GOODS_PRICE'].fillna(0, inplace = True) # 값을 무엇으로 채울지
    train['AMT_ANNUITY'].fillna(0, inplace = True) # 값을 무엇으로 채울지
    train['CNT_FAM_MEMBERS'].fillna(0, inplace = True) # 값을 무엇으로 채울지
    train['DAYS_LAST_PHONE_CHANGE'].fillna(0, inplace = True) # 값을 무엇으로 채울지   
    
    train.drop(['AMT_GOODS_PRICE','FLAG_EMP_PHONE','REGION_RATING_CLIENT_W_CITY', 'OBS_60_CNT_SOCIAL_CIRCLE',
               'CNT_CHILDREN','LIVE_REGION_NOT_WORK_REGION','LIVE_CITY_NOT_WORK_CITY', 'DEF_30_CNT_SOCIAL_CIRCLE'],axis=1,inplace=True)
    
    print(train.shape)
    
    # LabelEncode or One-hot Encoding
    
    features_label = []
    features_one_hot = []
    for i in train.columns:
        if train[i].dtype == 'object':
            if len(train[i].unique()) == 2:
                features_label.append(i)
            else:
                features_one_hot.append(i) 
    
    for feature in features_label:
        le = LabelEncoder()
        le = le.fit(train[feature])
        train[feature] = le.transform(train[feature])
        
    for feature in features_one_hot:
        college_one_hot_encoded = pd.get_dummies(train[feature], columns=[feature])
        train = pd.concat([train, college_one_hot_encoded], axis=1)
        
    print(train.shape)
    
    train.drop(features_one_hot, axis = 1, inplace = True)
    
    print(train.shape)
    
    train['new_XNA'] = np.array(train['XNA']).sum(axis = 1)
    train.drop(['XNA','F','M','Maternity leave', 'Unknown'], axis = 1, inplace = True)
    train.rename(columns = {'new_XNA' : 'XNA'}, inplace = True)
    
    return train

train = pd.read_csv('/kaggle/input/home-credit-default-risk/application_train.csv')
train = data_processing(train)

train.isnull().sum().sort_values(ascending = False)

# train_corr = train.corr()

# for i in train_corr.columns:
#     print(train_corr[(train_corr[i] > 0.8) | (train_corr[i] < -0.8)][i])
    
# # AMT_CREDIT & AMT_GOODS_PRICE : 0.98
# # DAYS_EMPLOYED & FLAG_EMP_PHONE : -0.99
# # REGION_RATING_CLIENT & REGION_RATING_CLIENT_W_CITY : 0.95
# # OBS_30_CNT_SOCIAL_CIRCLE & OBS_60_CNT_SOCIAL_CIRCLE : 0.99

# # CNT_FAM_MEMBERS & CNT_CHILDREN : 0.87
# # REG_REGION_NOT_WORK_REGION & LIVE_REGION_NOT_WORK_REGION : 0.86
# # REG_CITY_NOT_WORK_CITY & LIVE_CITY_NOT_WORK_CITY : 0.82
# # DEF_60_CNT_SOCIAL_CIRCLE & DEF_30_CNT_SOCIAL_CIRCLE : 0.86

# # 위 컬럼 중 하나 삭제

(307511, 65)
(307511, 57)
(307511, 158)
(307511, 150)


XNA                       0
Separated                 0
Group of people           0
Family                    0
Children                  0
                         ..
Culture                   0
Construction              0
Cleaning                  0
Business Entity Type 3    0
SK_ID_CURR                0
Length: 145, dtype: int64

### Data Anlaysis

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
import sklearn.metrics as metrics

In [12]:
def analysis(data_x, data_y):
    X_train, X_test, y_train, y_test=train_test_split(data_x, data_y, random_state=0)
 
    xgb_dtrain = xgb.DMatrix(data = X_train, label = y_train) # 학습 데이터를 XGBoost 모델에 맞게 변환
    xgb_dtest = xgb.DMatrix(data = X_test) # 평가 데이터를 XGBoost 모델에 맞게 변환
    
    xgb_param = {'max_depth': 3, # 트리 깊이
             'learning_rate': 0.01, # Step Size
             'n_estimators': 100, # Number of trees, 트리 생성 개수
             'objective': 'multi:softprob',
             'num_class': 2} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
    
    xgb_model = xgb.train(params = xgb_param, dtrain = xgb_dtrain) # 학습 진행
    xgb_model_predict = xgb_model.predict(xgb_dtest) # 평가 데이터 예측
    
    cnt = 1
    
#     for i in range(7,15,1):
#         for j in range(50,100,10):
#             cnt += 1
#             xgb_param['max_depth'] = i
#             xgb_param['n_estimators'] = j
#             xgb_model = xgb.train(params = xgb_param, dtrain = xgb_dtrain) # 학습 진행
#             xgb_model_predict += xgb_model.predict(xgb_dtest) # 평가 데이터 예측
    
    xgb_model_predict /= cnt
    xgb_model_predict = np.argmax(xgb_model_predict, axis = 1)
    print(roc_auc_score(y_test, xgb_model_predict))
    print(metrics.f1_score(y_test, xgb_model_predict)) # 정확도 % 계산
    
    return xgb_model_predict

train_y = train['TARGET']
train_X = train.drop('TARGET', axis = 1)

analysis(train_X, train_y)

# def final_analysis(X_train, y_train, X_test):
#     # X_train, X_test, y_train, y_test=train_test_split(data_x, data_y, random_state=0)
 
#     xgb_dtrain = xgb.DMatrix(data = X_train, label = y_train) # 학습 데이터를 XGBoost 모델에 맞게 변환
#     xgb_dtest = xgb.DMatrix(data = X_test) # 평가 데이터를 XGBoost 모델에 맞게 변환
    
    
#     xgb_param = {'max_depth': 4, # 트리 깊이
#              'learning_rate': 0.01, # Step Size
#              'n_estimators': 100, # Number of trees, 트리 생성 개수
#              'objective': 'multi:softprob',
#              'num_class': 2} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
    
#     xgb_model = xgb.train(params = xgb_param, dtrain = xgb_dtrain) # 학습 진행
#     xgb_model_predict = xgb_model.predict(xgb_dtest) # 평가 데이터 예측
    
#     cnt = 1
    
#     for i in range(5,10,1):
#         for j in range(50,100,10):
#             cnt += 1
#             xgb_param['max_depth'] = i
#             xgb_param['n_estimators'] = j
#             xgb_model = xgb.train(params = xgb_param, dtrain = xgb_dtrain) # 학습 진행
#             xgb_model_predict += xgb_model.predict(xgb_dtest) # 평가 데이터 예측
    
#     xgb_model_predict /= cnt
#     xgb_model_predict = np.argmax(xgb_model_predict, axis = 1)
#     # print(xgb_model_predict)
#     # print("Accuracy: %.2f" % (accuracy_score(y_test, xgb_model_predict) * 100), "%") # 정확도 % 계산
    
#     return xgb_model_predict

# titanic_test = pd.read_csv('/kaggle/input/titanic/test.csv')

# submission = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

# pred1 = final_analysis(X_titanic_df, y_titanic_df, data_processing(titanic_test))

[17:59:55] WARNING: /workspace/src/learner.cc:328: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


0.5
0.0


array([0, 0, 0, ..., 0, 0, 0])

In [13]:
# def data_analysis_final(data_x, data_y):
#     X_train, X_test, y_train, y_test=train_test_split(data_x, data_y,random_state=0)
    
#     gb_clf = GradientBoostingClassifier(random_state=0, max_depth = 4, learning_rate=0.01, n_estimators = 100)
    
#     gb_clf1 = GradientBoostingClassifier(random_state=1, max_depth = 4, learning_rate=0.01, n_estimators = 100)
    
#     xgb_clf_1 = XGBClassifier(random_state=0, objective = 'binary:logistic',eval_metric="rmse", learning_rate=0.01, max_depth=5, n_estimators = 100)
    
#     xgb_clf_2 = XGBClassifier(random_state=1, objective = 'binary:logistic',eval_metric="rmse", learning_rate=0.01, max_depth=4, n_estimators = 50)
    
#     xgb_clf_3 = XGBClassifier(random_state=2, objective = 'binary:logistic',eval_metric="rmse", learning_rate=0.01, max_depth=4, n_estimators = 100)
    
#     # voting : 0.87 / rf, xgb
#     vo_clf = VotingClassifier(estimators=[ # ('dt',dt_clf),
#                                            # ('lr',lr_clf),
#                                            # ('knn',knn_clf),
#                                            ('gb',gb_clf),
#                                            ('gb1',gb_clf1),
#                                            ('xgb1',xgb_clf_1),
#                                            ('xgb2',xgb_clf_2),
#                                            ('xgb3',xgb_clf_3)
#     ]
#                               ,voting='soft')
#     vo_clf.fit(X_train , y_train)
    
#     pred = vo_clf.predict(X_test)
    
#     return pred

# data_analysis_final(train_X, train_y)

In [14]:
def data_processing_test(train):
    null_columns = sorted(list(train.isnull().sum().sort_values(ascending = False).index[:57]))

    new_columns = []

    for i in train.columns:
        if i not in null_columns:
            new_columns.append(i)
    
    train = train[new_columns]
    
    print(train.shape)
    
    train['NAME_TYPE_SUITE'].fillna('NaN', inplace = True)
    train['DEF_60_CNT_SOCIAL_CIRCLE'].fillna(0, inplace = True)
    train['OBS_60_CNT_SOCIAL_CIRCLE'].fillna(0, inplace = True)
    train['DEF_30_CNT_SOCIAL_CIRCLE'].fillna(0, inplace = True)
    train['OBS_30_CNT_SOCIAL_CIRCLE'].fillna(0, inplace = True)
    
    train['EXT_SOURCE_2'].fillna(0, inplace = True) # 값을 무엇으로 채울지
    train['AMT_GOODS_PRICE'].fillna(0, inplace = True) # 값을 무엇으로 채울지
    train['AMT_ANNUITY'].fillna(0, inplace = True) # 값을 무엇으로 채울지
    train['CNT_FAM_MEMBERS'].fillna(0, inplace = True) # 값을 무엇으로 채울지
    train['DAYS_LAST_PHONE_CHANGE'].fillna(0, inplace = True) # 값을 무엇으로 채울지   
    
    train.drop(['AMT_GOODS_PRICE','FLAG_EMP_PHONE','REGION_RATING_CLIENT_W_CITY', 'OBS_60_CNT_SOCIAL_CIRCLE',
               'CNT_CHILDREN','LIVE_REGION_NOT_WORK_REGION','LIVE_CITY_NOT_WORK_CITY', 'DEF_30_CNT_SOCIAL_CIRCLE'],axis=1,inplace=True)
    
    print(train.shape)
    
    # LabelEncode or One-hot Encoding
    
    features_label = []
    features_one_hot = []
    for i in train.columns:
        if train[i].dtype == 'object':
            if len(train[i].unique()) == 2:
                features_label.append(i)
            else:
                features_one_hot.append(i) 
    
    for feature in features_label:
        le = LabelEncoder()
        le = le.fit(train[feature])
        train[feature] = le.transform(train[feature])
        
    for feature in features_one_hot:
        college_one_hot_encoded = pd.get_dummies(train[feature], columns=[feature])
        train = pd.concat([train, college_one_hot_encoded], axis=1)
        
    print(train.shape)
    
    train.drop(features_one_hot, axis = 1, inplace = True)
    train.drop('CODE_GENDER', axis = 1, inplace = True)
    
    print(train.shape)
    
    return train

test = pd.read_csv('/kaggle/input/home-credit-default-risk/application_test.csv')
test = data_processing_test(test)

(48744, 64)
(48744, 56)
(48744, 152)
(48744, 144)


In [15]:
def analysis_test(X_train, y_train, test):
    # X_train, X_test, y_train, y_test=train_test_split(data_x, data_y, random_state=0)7
 
    xgb_dtrain = xgb.DMatrix(data = X_train, label = y_train) # 학습 데이터를 XGBoost 모델에 맞게 변환
    xgb_dtest = xgb.DMatrix(data = test) # 평가 데이터를 XGBoost 모델에 맞게 변환
    
    xgb_param = {'max_depth': 3, # 트리 깊이
             'learning_rate': 0.01, # Step Size
             'n_estimators': 100, # Number of trees, 트리 생성 개수
             'objective': 'multi:softprob',
             'num_class': 2} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
    
    xgb_model = xgb.train(params = xgb_param, dtrain = xgb_dtrain) # 학습 진행
    xgb_model_predict = xgb_model.predict(xgb_dtest) # 평가 데이터 예측
    
    cnt = 1
    
#     for i in range(7,15,1):
#         for j in range(50,100,10):
#             cnt += 1
#             xgb_param['max_depth'] = i
#             xgb_param['n_estimators'] = j
#             xgb_model = xgb.train(params = xgb_param, dtrain = xgb_dtrain) # 학습 진행
#             xgb_model_predict += xgb_model.predict(xgb_dtest) # 평가 데이터 예측
    
#     xgb_model_predict /= cnt
#     xgb_model_predict = np.argmax(xgb_model_predict, axis = 1)
#     print(roc_auc_score(y_test, xgb_model_predict))
#     print(metrics.f1_score(y_test, xgb_model_predict)) # 정확도 % 계산
    
    return xgb_model_predict

# test = data_processing_test(test)
result = analysis_test(train_X, train_y, test)
result = result[:,1]

[18:00:08] WARNING: /workspace/src/learner.cc:328: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [16]:
result[result > 0.46] = 1
result[result <= 0.46] = 0

In [17]:
submission = pd.read_csv('/kaggle/input/home-credit-default-risk/sample_submission.csv')

submission['TARGET'] = result

submission.to_csv('submission.csv', index=False)